<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8m_Focal_320_NoAugmentation_TruePretrained_052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8-m Classification @ 320px on Colab
No Augmentation!

REAL Focal with lower alpha and gamma!!

Pretrained = True..

In [1]:
# @title 1️⃣ Install dependencies
# Colab comes with torch+cuda preinstalled, but we upgrade ultralytics.
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:00


In [2]:
# @title 2️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title 3️⃣ Paths & Unzip Dataset
import os, zipfile, glob
from pathlib import Path

# Adjust these to match where you put your ZIP in Drive:
ZIP_PATH     = "/content/drive/MyDrive/DMS_Mixed_NotAugmented_320.zip"
EXTRACT_DIR  = "/content/dataset320"
SPLIT_DIR    = os.path.join(EXTRACT_DIR, "Mixed_SynDD2_100Data_Rearview_grayscale_320")  # must contain train/ & val/
RESULTS_DIR  = "/content/drive/MyDrive/yolov8m_cls_320_results"

# 3.1 Unzip if needed
if not os.path.isdir(SPLIT_DIR):
    print("📦 Unzipping dataset…")
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(EXTRACT_DIR)
    print("✅ Unzipped to", EXTRACT_DIR)

# 🔧 Split the flat folder of 14 classes into train/ and val/ sets
import os, random, shutil, glob
from pathlib import Path

# --- config -----------------------------------------------------------
SPLIT_DIR   = Path('/content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320')  # where the 14 class dirs are
TRAIN_RATIO = 0.90                                   # 90‑10 split
seed        = 42                                     # so it’s repeatable
# ----------------------------------------------------------------------

random.seed(seed)

train_root = SPLIT_DIR / 'train'
val_root   = SPLIT_DIR / 'val'
train_root.mkdir(exist_ok=True)
val_root.mkdir(exist_ok=True)

cls_dirs = sorted(
   [d for d in SPLIT_DIR.iterdir() if d.is_dir() and d.name not in ('train','val')],
   key=lambda d: d.name
)
assert len(cls_dirs) == 14, f"Expected 14 class folders, found {len(cls_dirs)}"

for cls_dir in cls_dirs:
    images = sorted(glob.glob(str(cls_dir / '*')))        # jpg / png / etc.
    random.shuffle(images)
    split_idx = int(len(images) * TRAIN_RATIO)

    # prepare destination folders
    (train_root / cls_dir.name).mkdir(parents=True, exist_ok=True)
    (val_root   / cls_dir.name).mkdir(parents=True, exist_ok=True)

    # move (or copy) the files
    for i, img_path in enumerate(images):
        dest_root = train_root if i < split_idx else val_root
        shutil.move(img_path, dest_root / cls_dir.name / Path(img_path).name)

    # optional: remove the now‑empty original class folder
    cls_dir.rmdir()

    print(f"{cls_dir.name:<20} → {split_idx:5d} train | {len(images)-split_idx:5d} val")

print("\n✅  Dataset now has the required .../train/ and .../val/ structure")


# 3.2 Verify
assert os.path.isdir(os.path.join(SPLIT_DIR, "train")), "❌ train/ missing!"
assert os.path.isdir(os.path.join(SPLIT_DIR, "val"  )), "❌ val/   missing!"

train_images = glob.glob(os.path.join(SPLIT_DIR, "train", "*", "*.jpg"))
val_images   = glob.glob(os.path.join(SPLIT_DIR, "val",   "*", "*.jpg"))
print(f"🧾 Found {len(train_images):,} train and {len(val_images):,} val images")

📦 Unzipping dataset…
✅ Unzipped to /content/dataset320
Control_Panel        →  8507 train |   946 val
Drinking             →  7790 train |   866 val
Eating               →  2159 train |   240 val
Makeup               →  6196 train |   689 val
Normal               →  7975 train |   887 val
Phone_Call_(Left)    →  7125 train |   792 val
Phone_Call_(Right)   →  7437 train |   827 val
Reaching_Behind      →  4633 train |   515 val
Sleep                →  4450 train |   495 val
Smoking              → 12368 train |  1375 val
Talk_to_passengers   →  5335 train |   593 val
Text_(Left)          →  5982 train |   665 val
Text_(Right)         →  5780 train |   643 val
Yawning              →  3265 train |   363 val

✅  Dataset now has the required .../train/ and .../val/ structure
🧾 Found 89,002 train and 9,896 val images


In [4]:
# @title 4️⃣ Compute α (class imbalance) for focal loss
import torch

# original per‑class counts
# after splitting, count only train images per class
class_counts = {}

for cls_folder in sorted((SPLIT_DIR/"train").iterdir(), key=lambda d: d.name):
    class_counts[cls_folder.name] = len(list(cls_folder.glob('*')))

# then compute alpha as before
max_n       = max(class_counts.values())
raw_alpha   = torch.tensor([max_n / class_counts[c] for c in class_counts], dtype=torch.float32)
alpha_clipped = torch.minimum(raw_alpha, torch.tensor(2.2))  # cap at 2.2 if you like
alpha       = alpha_clipped.cuda()

print("α weights (train only):", alpha)

α weights (train only): tensor([1.4539, 1.5877, 2.2000, 1.9961, 1.5508, 1.7359, 1.6630, 2.2000, 2.2000,
        1.0000, 2.2000, 2.0675, 2.1398, 2.2000], device='cuda:0')


In [5]:
# @title 5️⃣ Define FocalLoss & Train YOLOv8-m-cls
import torch.nn as nn
from ultralytics import YOLO

# ─── Focal Loss ──────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=1.5, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma, self.alpha, self.reduction = gamma, alpha, reduction

    def forward(self, inputs, targets):
        logp = torch.nn.functional.log_softmax(inputs, dim=-1)
        p    = torch.exp(logp)
        t    = targets.long()
        pt   = p.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        focal = (1 - pt) ** self.gamma
        loss = -focal * logp.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        if self.alpha is not None:
            loss = loss * self.alpha[t]
        return loss.mean() if self.reduction=='mean' else loss.sum()

# ─── Load & override loss ─────────────────
print("🚀 Loading YOLOv8m-cls…")
model = YOLO("yolov8m-cls.pt")

# model = YOLO('/content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal/weights/last.pt')             # pretrained backbone

model.model.loss_fn = FocalLoss(gamma=1.5, alpha=alpha)
print("🔍 Active loss_fn:", model.model.loss_fn)

# ─── Train ───────────────────────────────
print("📚 Training…")
model.train(
    data=SPLIT_DIR,          # train/ & val/ subfolders
    epochs=50,
    batch=32,
    imgsz=320,
    fliplr=0,         # <-- no left‑right flips for any image
    flipud=0,         # <-- no up‑down flips for any image
    project=RESULTS_DIR,
    name="yolov8m_cls_320_focal",
    pretrained=True,
    lr0=5e-4,
    optimizer="Adam",
)

print("✅ Training resumed!")

# ─── Validate & copy best.pt ────────────
print("📊 Validating…")
model.val()

src = os.path.join(model.trainer.save_dir, "weights/best.pt")
dst = os.path.join(RESULTS_DIR, "yolov8m_cls_320_focal", "weights/best.pt")
os.makedirs(os.path.dirname(dst), exist_ok=True)
os.system(f"cp {src} {dst}")
print("✅ Best weights saved to:", dst)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading YOLOv8m-cls…


100%|██████████| 32.7M/32.7M [00:00<00:00, 124MB/s]


🔍 Active loss_fn: FocalLoss()
📚 Training…
Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0, flipud=0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_cls_320_focal, nbs=64, nms=False, opset=None, optimize=False, opt

100%|██████████| 5.35M/5.35M [00:00<00:00, 98.1MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 937.9±365.7 MB/s, size: 24.0 KB)


...


50 epochs completed in 3.347 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal/weights/last.pt, 31.7MB
Optimizer stripped from /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal/weights/best.pt, 31.7MB

Validating /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal/weights/best.pt...
Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,780,590 parameters, 0 gradients, 41.6 GFLOPs
train: /content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320/train... found 89002 images in 14 classes ✅ 
val: /content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320/val... found 9896 images in 14 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 155/155 [00:08<00:00, 19.05it/s]


                   all      0.993          1
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal
✅ Training resumed!
📊 Validating…
Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,780,590 parameters, 0 gradients, 41.6 GFLOPs
train: /content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320/train... found 89002 images in 14 classes ✅ 
val: /content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320/val... found 9896 images in 14 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 746.3±290.5 MB/s, size: 24.2 KB)


val: Scanning /content/dataset320/Mixed_SynDD2_100Data_Rearview_grayscale_320/val... 9896 images, 0 corrupt: 100%|██████████| 9896/9896 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 310/310 [00:13<00:00, 22.87it/s]


                   all      0.993          1
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal2
✅ Best weights saved to: /content/drive/MyDrive/yolov8m_cls_320_results/yolov8m_cls_320_focal/weights/best.pt


In [ ]:
import os
import time
from google.colab import runtime

# Wait for training to complete before disconnecting
print("Training completed! Releasing GPU resources...")
time.sleep(20)  # Give some time for final processing

# Automatically disconnect the Colab session
runtime.unassign()
os._exit(0)

Training completed! Releasing GPU resources...
